# Machine Translation

In diesem Notebook möchten wir uns mit der Königsdisziplin des Natural Language Processings beschäftigen, der maschinellen Übersetzung.
Vermutlich gibt es keine NLP-Anwendung, die einerseits vielen bekannt ist, anderseits durch Deep Learning und neuronalen Netzen einen solchen Aufschwung bekommen hat.
Vor dem Siegeszug der neuronalen Netzen, wurde maschinelle Übersetzung deshalb als schwierig angesehen, weil sie alle Teilaspekte von Sprache beinhaltet. Neben grammatikalischer Korrektheit, sollen maschinell übersetzte Texte den Sinn des Originaltextes wiedergeben und darüberhinaus auch den Subtext, wie Ironie, Witz, erfassen.
Ob letzteres einfach gelingt, sei hier mal dahingestellt.

## Daten
Im Vergleich zu den bisherigen Notebooks sind die Daten für machine translation etwas dröge. Daher möchten wir uns hier nicht allzu lange aufhalten. Speichert zunächst folgende Datei ab und entpackt sie: http://www.manythings.org/anki/fra-eng.zip

Danach wollen wir die Daten einlesen. Das Format ist einfach, pro Zeile steht ein Satzpaar getrennt von einem Tab.

In [ ]:
with open('fra.txt', 'r') as f:
    lines = f.readlines()

In [ ]:
pairs = [tuple(l.split("\t")) for l in lines]

In [ ]:
pairs_cleaned = [(e, f.strip()) for e, f in pairs]

Damit unser Modell weiß wo Anfang und Ende der französischen Sätze sind, müssen wir noch ein spezielles Start- (🏳️) und Endsymbol (🏴) einfügen:

In [ ]:
pairs_cleaned_with_marker = [(e, "🏳️" + f + "🏴") for e, f in pairs_cleaned]

Speichert euch nun alle Character, die in den englischen Sätzen und in den französischen Sätzen in jeweils einem Set ab.

In [ ]:
english_characters = set()
french_characters = set()
# TODO

Um unser Modell später mit passenden Eingabedaten füttern zu können, müssen wir nun noch die jeweiligen Vektorlängen für den encoder und den decoder bestimmen. Speichert die Längen in den Variablen  `encoder_vector_len` und `decoder_vector_len` ab. Hinweis: uns Modell bekommt Vektoren, die die Characters pro Wort abbilden als Eingabe.

In [ ]:
encoder_vector_len = #TODO
decoder_vector_len = #TODO
print(encoder_vector_len)
print(decoder_vector_len)

Um die Eingabevektoren bestimmen zu können, müssen wir uns noch Dictionaries bauen, die jeweils die Character auf die Komponenten des Vektors abbilden

In [1]:
english_token_index = {} #TODO 
french_token_index = {} #TODO

Wir haben nun alle Dimensionen bestimmt, um die Trainingsdaten zu definieren. Dafür bauen wir uns drei Matrizen (eine als Eingabe für den Encoder, eine als Eingabe für den Decoder und eine als Ausgabe des Decoders) zusammen. Die Dimensionen dafür sind jeweils: Anzahl Sätze, Anzahl Wörter des längsten Satzes, Anzahl Characters.

In [ ]:
import numpy as np

max_encoder_sentence_len = max((len(e) for e, _ in pairs_cleaned_with_marker))
max_decoder_sentence_len = max((len(f) for _, f in pairs_cleaned_with_marker))


encoder_input_data = np.zeros(
    (len(pairs_cleaned_with_marker), max_encoder_sentence_len, encoder_vector_len),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(pairs_cleaned_with_marker), max_decoder_sentence_len, decoder_vector_len),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(pairs_cleaned_with_marker), max_decoder_sentence_len, decoder_vector_len),
    dtype='float32')

Befüllt nun die Matrizen. Die `encoder_input_data` und die `decoder_input_data`-Matrix soll für jeden Satz aus den Trainingsdaten, einen Eintrag mit einem Token-Vektor pro Wort enthalten.
Die `decoder_target_data`-Matrix soll den gleichen Inhalt wie die `decoder_input_data`-Matrix haben, allerdings um eine Position verschoben. (Schaut euch nochmal die Folien an, wir brauchen Platz für das Startsymbol) 

Die `encoder_input_data`-Matrix wird mit den englischen Daten befüllt, die beiden anderen mit den französischen Daten.



In [ ]:
for i, (e, f) in enumerate(pairs_cleaned_with_marker):
    for t, char in enumerate(e):
        encoder_input_data[i, t,#TODO] = 1.
    for t, char in enumerate(f):
        decoder_input_data[i, t, #TODO] = 1.
        if t > 0:
            decoder_target_data[i, #TODO, #TODO] = 1.

## Sequence-to-sequence Modelle

Im Gegensatz zu den bisherigen Notebooks, wollen wir hier mit dem Modell beginnen. Einerseits, weil die Daten eher dröge sind (englisch-fränzosische Satzpaare), andererseits, weil das Modell selbst komplizierter ist und wir hier die Functional-API von Keras benutzen müssen.

Im Gegensatz zu der Sequence-API, definiert man hier ein Modell in dem, den jeweiligen Layer mit dem Vorgänger Layer aufruft:
```

a = Input(...)
b = Dense(...)

t = b(a)


```
Als Rückgabe bekommt man einen Tensor.


Ganz generell gesprochen, möchten wir hier ein Seq2Seq-Modell aufbauen, das auf LSTMs basiert. Wie auf den Folien beschrieben wurde, teilt sich so ein Modell in einen Encoder und Decoder auf. 
Wir beginnen die Implementierung mit dem Encoder.

### Encoder

Auch das komplexeste Modell fängt mit einer Eingabe ein.

Definiert einen Input-Layer, der als Eingabedimension, Sätze beliebiger Länge als Vektoren nimmt. Eine Komponente im Vektor repräsentiert einen Character. Unbekannte Teile der Eingabedimensionen kann man mit `None` definieren.

In [ ]:
from keras.layers import Input

encoder_input = Input(shape=#TODO)


Als nächstes definiert einen LSTM-Layer, der 256 hidden units hat. Die Ausgabe des Layers soll später an den Decoder 'verfüttert' werden. Seht in der Dokumentation nach wie man den Thought-Vektor zurückbekommen kann (https://keras.io/layers/recurrent/)

In [ ]:
from keras.layers import LSTM

encoder = LSTM(#TODO)

Verbindet nun den `encoder_input`-Layer mit dem `encoder`-Layer

In [ ]:
_, state_h, state_c = #TODO

Der Encoder wäre damit fertig implementiert.

### Decoder

Auch der Decoder fängt mit einem Input-Layer an. Definiert einen Input-Layer analog zum Encoder.

In [ ]:
decoder_input = Input(shape=#TODO)

Wie auch im Encoder, ist das Herzstück ein LSTM-Layer. Definiert einen LSTM-Layer mit 256 hidden units. Seht in der Dokumentation nach, wie man neben den Thought-Vektoren, auch die komplette Ausgabe-Sequence zurückbekommt.

In [ ]:
decoder_lstm = LSTM(#TODO)

Verbindet nun den `decoder_input`-Layer mit dem `decoder_lstm`-Layer. An welcher der drei Ausgaben sind wir hier interessiert? Außerdem müssen wir hier, den Thought-Vektor des Encoders mit übergeben, dies geschieht über den `initial_state`-Parameter

In [ ]:
?,?,? = decoder_lstm(#TODO)

Den Abschluss unseres Modells bildet ein Dense-Layer, der gleich viele `hidden_units` wie der Input-Layer des Decoders hat. Als Aktivierungsfunktion nehmen wir die Softmax-Funktion.

In [ ]:
from keras.layers import Dense

decoder_layer = Dense(#Todo)

Verbindet diesen Layer mit dem bisherigen Decoder.

In [ ]:
decoder = # TODO

Wir haben nun alle Elemente für ein Sequence-To-Sequence-Modell zusammen. Im letzten Schritt bauen wir das ganze Modell zusammen.

In [ ]:
from keras import Model

model = Model(inputs=[encoder_input, decoder_input], outputs=[decoder])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

Fertig! Nun kann das Modell trainiert werden.

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=100,
          validation_split=0.2)

### Inferenz

Um mit dem trainierten Modell Texte übersetzen zu können, müssen wir nun noch unser bisheriges Modell etwas umbauen.
Im ersten Schritt definieren wir uns ein `Model` das einen Eingabetext encodiert. Als Eingabe hat dieses `Model` den bisherigen `encoder_input`-Layer und die Thought-Vektoren als Output. 


In [ ]:
encoder_model = Model(encoder_input, [state_h, state_c])

Etwas komplizierter ist das Decoder-Modell. Als Input nimmt es zum einen die Thought-Vektoren des Encoders, zum anderen den `decoder_input`-Layer von oben.
Definiert zunächst zwei Input-Layer für die Thought-Vektoren.

In [ ]:
decoder_state_input_h = Input(shape=(#TODO))
decoder_state_input_c = Input(shape=(#TODO))

Setzt diese beiden Input-Layer als `initial_state` und den `decoder_input`-Layer als Input in das `decoder_lstm` von oben ein.

In [ ]:
decoder_outputs, state_h, state_c = decoder_lstm(decoder_input, initial_state=[decoder_state_input_h, decoder_state_input_c])

Setzt nun den `decoder_outputs`-Tensor in den `decoder_layer` von oben ein.

In [ ]:
decoder_outputs = #TODO

Nun können wir ein `Model` zusammenbauen, das uns als Decoder dient. Input ist hier der `decoder_input`-Layer, zusammen mit den `decoder_state_input_h` und dem `decoder_state_input_c`-Tensor. Als Output dient uns der `decoder_outputs`-Tensor und der `state_h` mit dem `state_c`-Tensor. 

In [ ]:
decoder_model = Model(#TODO, #TODO)

Ein kleiner Zwischenschritt ist noch nötig, bevor wir mit dem Modell Texte übersetzen können. Damit wir aus unseren Vektoren wieder Texte bekommen, müssen wir noch das Dictionary `reverse_french_token_index` befüllen, das die "Umkehrung" von `french_token_index` ist.

In [ ]:
reverse_french_token_index = {} #TODO

Nun können wir eine Funktion `translate` implementieren, die als Eingabe einen englischen Text als Vektoren nimmt.

In [ ]:
def translate(english_text):
    # Wandelt mit dem encoder_model, `english_text` in einen Thought-Vektor um
    thought = encoder_model.predict(english_text)
    
    # Als Input für den Decoder, wir zum einen der thought-Vektor benötigt, 
    # zum anderen die Vektor-Repräsentation des Startsymbols
    french_text = np.zeros((1, 1, #TODO))
    
    # Belegt die Komponente des Start-Symbols mit 1
    french_text[0, 0, #TODO] = 1.
    
    # und übergebt beides an den Decoder
    chars, h, c = decoder_model.predict([french_text] + thought)
    
    # chars beinhaltet eine 1 x 1 x decoder_vector_len Matrix, die für jeden französische Character
    # eine Art "Wahrscheinlichkeit" angibt.
    # Findet die Komponente, mit der größten Wahrscheinlichkeit und wandelt sie in ein Character um
    french_char_idx = #TODO
    
    french_char = reverse_french_token_index[french_char_idx]
    
    # die nächste Prediction wäre dann
    french_text = np.zeros((1, 1, decoder_vector_len))
    french_text[0, 0, french_char_idx] = 1.
    chars, h, c = decoder_model.predict([french_text, h, c])
    
    french_char_idx = #TODO
    
    french_char = reverse_target_char_index[french_char_idx]
    
    # baut eine Schleife mit geeigneter Abbruchbedingung, die die Schritte von oben umfasst 
    # und dabei aus french_char einen übersetzen Text zusammenbaut
    return french_char

In [ ]:
english_text = encoder_input_data[10] 
french_text = translate(english_text)

print(pairs_cleaned_with_marker[10])
print(french_text)


### Hausaufgabe: 
 Implementiert die BLEU-Metrik wie auf den Folien nach und evaluiert euer Modell damit.